In [9]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

# First Part

## Scraping from wikipedia

In [10]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table') 


In [11]:
df = pd.read_html(str(table))[0]
df.columns = ['PostalCode', 'Borought', 'Neighborhood']
df = df[1:]
print("Shape : ", df.shape)
df.head()

Shape :  (287, 3)


,PostalCode,Borought,Neighborhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


In [12]:
# Removing data where Borought is not assigned
df = df[df.Borought != 'Not assigned']
df.shape

(210, 3)

## Creating the DataFrame

"More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table."

In [13]:
postCode = list(np.unique(df['PostalCode'].values))

In [14]:
neigbList = []
borought = []
for pc in postCode:
    neigbList.append((df[df.PostalCode==pc]['Neighborhood']).str.cat(sep =", "))
    borought.append(df[df.PostalCode == pc]['Borought'].iloc[0])
pd.Series(neigbList[:5])

0                            Rouge, Malvern
1    Highland Creek, Rouge Hill, Port Union
2         Guildwood, Morningside, West Hill
3                                    Woburn
4                                 Cedarbrae
dtype: object

In [15]:
frame = { 'PostalCode': pd.Series(postCode), 'Borought': pd.Series(borought),'Neighborhood':pd.Series(neigbList) } 
new_df = pd.DataFrame(frame)
new_df = new_df[['PostalCode','Borought','Neighborhood']]
new_df.head()

,PostalCode,Borought,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [16]:
for index, row in new_df.iterrows(): 
    if (row["Neighborhood"] == 'Not assigned'):
        new_df['Neighborhood'].replace('Not assigned', row['Borought'], inplace=True)
        


In [17]:
new_df[new_df.PostalCode=='M7A']

,PostalCode,Borought,Neighborhood
85,M7A,Queen's Park,Queen's Park


In [18]:
new_df.shape

(103, 3)

# Second Part

In [20]:
!wget -O Geospatial_data.csv 

/bin/sh: wget: command not found


In [23]:
geo_df = pd.read_csv("http://cocl.us/Geospatial_data")
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [28]:
merged_df= pd.merge(left=new_df,right=geo_df, left_on='PostalCode', right_on='Postal Code')
print(merged_df.shape)
merged_df.head()

(103, 6)


,PostalCode,Borought,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476
